<a href="https://colab.research.google.com/github/raja-19/julia-gpu-task/blob/main/julia_gpu_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
using Pkg
Pkg.add("BenchmarkTools")
Pkg.add("Plots")

In [ ]:
using CUDA
using BenchmarkTools
using Plots
using Test

For simplicity, I used `arr1` whose vectors and matrices are filled with 1's and `arr2` with 2's.

In [ ]:
k = 10
n = 2
arr1 = fill((fill(1.0f0, n), fill(1.0f0, n, n)), k)
arr2 = fill((fill(2.0f0, n), fill(2.0f0, n, n)), k)

#CPU

First I tried to perform the computation on the CPU. Julia's broadcasting and `map` allow to do that in an elegant way.

In [ ]:
arr3 = map(.+, arr1, arr2)

Julia packages provide a convenient way to do basic unit testing

In [ ]:
expected = fill((fill(3.0f0, n), fill(3.0f0, n, n)), k)
@test arr3 == expected

...and benchmarking.

In [ ]:
@btime map(.+, $arr1, $arr2)
nothing

#GPU

Element-wise addition is a simple operation and can be handled by CUDA.jl without the need to write custom kernels. The only challenge is to transform the arrays into a form suitable to be sent to the GPU. Data has to reside in a contiguous block of memory before being sent. I "flatten" the inital arrays into 2D arrays of `Float32`'s using `mapreduce` and `hcat` before the computation and go back to the original format after the computation. The postfixes `_h` stands for "host" (CPU), and `_d` - for "device" (GPU).

In [ ]:
arr1_h = mapreduce(tup->hcat(tup[1], tup[2]), hcat, arr1)
arr2_h = mapreduce(tup->hcat(tup[1], tup[2]), hcat, arr2)
arr1_d = CuArray(arr1_h)
arr2_d = CuArray(arr2_h)
arr3_d = arr1_d + arr2_d
arr3_h = Array(arr3_d)
arr3 = [(arr3_h[:,i], arr3_h[:,i+1:i+n]) for i in 1:(n+1):size(arr3_h, 2)]

In [ ]:
expected = fill((fill(3.0f0, n), fill(3.0f0, n, n)), k)
@test arr3 == expected

In [ ]:
@btime CUDA.@sync $arr1_d + $arr2_d
nothing

For large enough $k$ and $n$, the compuation on the GPU is significantly faster than on the CPU. This is not a fair comparison since the array preprocessing and postprocessing as well as the data transfer are not accounted for. With these factors taken into account, a single element-wise addition is more performant on the CPU.

#Exploiting symmetricity on the host

I tried to exploit the fact that the matrices in the arrays are symmetric (as covariance matrices). Thus, the addition can be performed only for their upper triangle parts. The strategy here is to flatten the matrices into 1D arrays using only their upper triangle parts (example below), combine them and the mean vectors into one long array and use that for addition on the GPU.
```
1 2 3
  4 5  ---->  1 2 3 4 5 6
    6
```

In [ ]:
function sym2flat(S, n)
  @inbounds [S[i, j] for i in 1:n for j in i:n]
end

function flat2sym(a, n)
  S = Array{Float32}(undef, n, n)
  for i in 1:n
    for j in i:n
      flatIdx = (i-1)*(2*n-i)÷2+j
      @inbounds S[i, j] = S[j, i] = a[flatIdx]
    end
  end
  S
end

In [ ]:
arr1_h = mapreduce(tup->vcat(tup[1], sym2flat(tup[2], n)), vcat, arr1)
arr2_h = mapreduce(tup->vcat(tup[1], sym2flat(tup[2], n)), vcat, arr2)
arr1_d = CuArray(arr1_h)
arr2_d = CuArray(arr2_h)
arr3_d = arr1_d + arr2_d
arr3_h = Array(arr3_d)
arr3 = [(arr3_h[1:n], flat2sym(arr3_h[n+1:n*(n+3)÷2], n)) for i in 1:n*(n+3)÷2:length(arr3_h)]

In [ ]:
expected = fill((fill(3.0f0, n), fill(3.0f0, n, n)), k)
@test arr3 == expected

In [ ]:
@btime CUDA.@sync $arr1_d + $arr2_d
nothing

#Exploiting symmetricity on the device



This part was done mostly to just practice writing a custom kernel. Now symmetric elements of a matrix are calculated by the same thread in a way that reduces the number of accesses to the global memory. The data layout is the same as the data layout in the first GPU approach. The implicit layout of the threads is the same as the data layout in the prevous approach. Because these two layouts do not match, careful index mapping is required.

In [ ]:
function add_sym!(arr1_d, arr2_d, arr3_d)
  n = size(arr3_d, 1)
  k = size(arr3_d, 2) ÷ (n + 1)
  start = (blockIdx().x - 1) * blockDim().x + threadIdx().x
  stride = blockDim().x * gridDim().x
  tupsize = n * (n + 3) ÷ 2
  for idx in start:stride:k*n*(n+3)÷2
    tupidx = (idx - 1) ÷ tupsize + 1
    tupoffset = (idx - 1) % tupsize + 1
    if tupoffset <= n
      i = tupoffset
      j = (tupidx - 1) * (n + 1) + 1
      @inbounds arr3_d[i, j] = arr1_d[i, j] + arr2_d[i, j]
    else
      matoffset = tupoffset - n
      mati = Int(floor((2*n+3-sqrt((2*n+3)^2-8*(n+matoffset)))/2))
      matj = matoffset-(mati-1)*(2*n-mati) ÷ 2
      i, j = mati, (tupidx - 1) * (n + 1) + 1 + matj
      invi, invj = matj, (tupidx - 1) * (n + 1) + 1 + mati
      @inbounds arr3_d[invi, invj] = arr3_d[i, j] = arr1_d[i, j] + arr2_d[i, j]
    end
  end
  nothing
end

In [ ]:
arr1_h = mapreduce(tup->hcat(tup[1], tup[2]), hcat, arr1)
arr2_h = mapreduce(tup->hcat(tup[1], tup[2]), hcat, arr2)
arr1_d = CuArray(arr1_h)
arr2_d = CuArray(arr2_h)
arr3_d = CuArray{Float32}(undef, n, k*(n+1))

nthreads = 1024
nblocks = ceil(Int, k*(n*(n+3)÷2)/nthreads)
CUDA.@sync @cuda threads=nthreads blocks=nblocks add_sym!(arr1_d, arr2_d, arr3_d)

arr3_h = Array(arr3_d)
arr3 = [(arr3_h[:,i], arr3_h[:,i+1:i+n]) for i in 1:(n+1):size(arr3_h, 2)]

In [ ]:
expected = fill((fill(3.0f0, n), fill(3.0f0, n, n)), k)
@test arr3 == expected

In [ ]:
@btime CUDA.@sync @cuda threads=nthreads blocks=nblocks add_sym!($arr1_d, $arr2_d, $arr3_d)

For large enough $k$ and $n$, the performance deteriorates even compared to the first GPU approach. The kernel might have become too heavy due to index mapping.

#Benchmarking and visualization

Benchmarking is performed for the most promising approach with exploiting symmetricity on the host. Since data preprocessing is quite heavy, the correct data layout in the GPU is hardcoded.

In [ ]:
function benchmark_wrapper(k, n)
  arr1_d = CUDA.fill(1.0f0, k*n*(n+3)÷2)
  arr2_d = CUDA.fill(2.0f0, k*n*(n+3)÷2)
  @belapsed CUDA.@sync $arr1_d + $arr2_d
end

In [ ]:
ns = [10, 20, 50, 100]
ks = [100, 200, 500, 1000]
t = [10^6 * benchmark_wrapper(k, n) for k in ks for n in ns]
nothing

In [ ]:
wireframe(ns, ks, t, xlabel="n", ylabel="k", zlabel="time, us")

In [ ]:
tmesh = reshape(t, length(ns), length(ks))

In [ ]:
klabels = labels=reshape(["k=$k" for k in ks], 1, :)
plot(ns, tmesh, marker=:diamond, xlabel="n", ylabel="time, us", labels=klabels)

In [ ]:
nlabels = labels=reshape(["n=$n" for n in ns], 1, :)
plot(ks, transpose(tmesh), marker=:diamond, xlabel="k", ylabel="time, us", labels=nlabels)

The plots indicate a linear growth with $k$ and a quadratic growth with $n$. Both are expected for large enough $k$ and $n$.